In [1]:
# Dependencies
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
import requests
from config import crimekey
from config import censuskey
from pprint import pprint
from census import Census

In [5]:
# API Setup
baseurl = "http://api.usa.gov/crime/fbi/sapi/"
c = Census(censuskey, year=2019)

In [6]:
URL = ("https://api.usa.gov/crime/fbi/sapi/api/summarized/state/CA/violent-crime/2020/2020?API_KEY=" + crimekey)

getdatasets = requests.get(URL)
datasets_json = getdatasets.json()
pprint(datasets_json)

{'pagination': {'count': 733, 'page': 0, 'pages': 37, 'per_page': 20},
 'results': [{'actual': 700,
              'cleared': 402,
              'data_range': None,
              'data_year': 2020,
              'offense': 'violent-crime',
              'ori': 'CA0010000',
              'state_abbr': 'CA'},
             {'actual': 193,
              'cleared': 61,
              'data_range': None,
              'data_year': 2020,
              'offense': 'violent-crime',
              'ori': 'CA0010100',
              'state_abbr': 'CA'},
             {'actual': 29,
              'cleared': 7,
              'data_range': None,
              'data_year': 2020,
              'offense': 'violent-crime',
              'ori': 'CA0010200',
              'state_abbr': 'CA'},
             {'actual': 537,
              'cleared': 140,
              'data_range': None,
              'data_year': 2020,
              'offense': 'violent-crime',
              'ori': 'CA0010300',
              'state

In [7]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {'for': 'state:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["State", "Name", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]

census_pd.head()

,State,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate
0,01,Alabama,4876250.0,39.0,50536.0,27928.0,795989.0,16.323794,2.708946
1,02,Alaska,737068.0,34.3,77640.0,36787.0,76933.0,10.437707,3.637114
2,04,Arizona,7050299.0,37.7,58945.0,30694.0,1043764.0,14.804535,2.778676
3,05,Arkansas,2999370.0,38.1,47597.0,26577.0,496260.0,16.545475,2.349860
4,06,California,39283497.0,36.5,75235.0,36955.0,5149742.0,13.109174,3.052765
